In [3]:
# idea : devlop a simple model can be the overall prediction benchmark 

# ref 
# https://github.com/yennanliu/movie_recommendation/blob/master/ref/Analysis_Algorithms_Movie%20Recommendation_Systems.pdf

In [13]:
"""

We denote the baseline prediction for user u and item i by bui.


The simplest baseline is to predict the average rating over all ratings in the system: bu;i = 
(where  is the overall average rating). This can be enhanced somewhat by predicting the
average rating by that user or for that item: bu;i = ru or bu;i = ru . Baselines can be further
enhanced by combining the user mean with the average deviation from user mean rating for a
particular item. Generally, a baseline predictor of the following form can be used:


"""

'\n\nWe denote the baseline prediction for user u and item i by bui.\n\n\nThe simplest baseline is to predict the average rating over all ratings in the system: bu;i = \x16\n(where \x16 is the overall average rating). This can be enhanced somewhat by predicting the\naverage rating by that user or for that item: bu;i = ru or bu;i = ru . Baselines can be further\nenhanced by combining the user mean with the average deviation from user mean rating for a\nparticular item. Generally, a baseline predictor of the following form can be used:\n\n\n'

In [6]:
# import library 


import pandas as pd 
from sklearn.cross_validation import train_test_split
import numpy as np
import time


In [168]:
def get_data():
    route='datasets/ml-latest-small/'
    #df_movie = pd.read_csv(route +'movies.csv')
    df_ratings = pd.read_csv(route +'ratings.csv')
    return df_ratings

def data_preprocess(df):
    df['view_count']=df.groupby(['userId','movieId']).count().reset_index()['rating']
    movie_grouped = df.groupby(['movieId']).agg({'view_count': 'count'}).reset_index()
    grouped_sum = movie_grouped['view_count'].sum()
    movie_grouped['percentage']  = movie_grouped['view_count'].div(grouped_sum)*100
    movie_grouped.sort_values(['view_count', 'movieId'], ascending = [0,1])
    return movie_grouped


def get_train_test_data(df):
	train_data, test_data = train_test_split(df, test_size = 0.20, random_state=0)
	return train_data, test_data


def get_rating_deviation(df):
    df_ = df.copy()
    userlist = list(set(df.userId))
    rating_divation = []
    for user in userlist:
        df_user = df[df.userId == user]
        #avg_user_rating = df_user.rating.mean()
        std_user_rating = df_user.rating.std()
        rating_divation.append(std_user_rating)
    df_std_rating = pd.DataFrame({'userId':userlist, 'std_rating': rating_divation })
    print (df_std_rating.head(10))
    #return df_std_rating
    # merge back 
    df_ = pd.merge(df_, df_std_rating, on='userId')
    return df_
    
        


def get_benchmark_feature(df):
	# total view 
	view_count_total = df.groupby(['movieId']).count().reset_index()[['movieId','rating']]
	view_count_total.columns = ['movieId','total_view']
	# avg rating 
	df_avg_rating = df.groupby(['movieId']).mean().reset_index()[['movieId','rating']]
	df_avg_rating.columns = ['movieId','avg_rating']
	# add average rating deviation
    
	# merge 
	df_ratings_viewcount = pd.merge(df_ratings, view_count_total, on='movieId')
	df_ratings_viewcount = pd.merge(df_ratings_viewcount, df_avg_rating, on='movieId')
	return df_ratings_viewcount
 
# -------------------------------------
# model 


def recommend(df_feature):
	# get movie with top view counts and avg_rating 
	recommend_list = df_ratings_viewcount.groupby('movieId')\
                                     .mean()[['total_view','avg_rating']]\
                                     .sort_values(['total_view','avg_rating'],ascending=False)\
                                     .reset_index()
	# return top 20 movie as recommendation 
	recommend_list_ = recommend_list.head(20) 
	print ('recommend list  : ')
	print (recommend_list_)
	return recommend_list_


def recommend_by_user(df_feature):
	pass 


In [169]:
df_std_rating = get_rating_deviation(df_ratings)

   std_rating  userId
0    0.887041       1
1    0.901753       2
2    0.741752       3
3    0.947616       4
4    0.565775       5
5    1.241195       6
6    0.934017       7
7    0.794978       8
8    0.883062       9
9    0.915777      10


In [172]:
#df_std_rating

In [166]:
#df_std_rating

pd.merge(df_ratings, df_std_rating, on='userId')

,userId,movieId,rating,timestamp,std_rating
0,1,31,2.5,1260759144,0.887041
1,1,1029,3.0,1260759179,0.887041
2,1,1061,3.0,1260759182,0.887041
3,1,1129,2.0,1260759185,0.887041
4,1,1172,4.0,1260759205,0.887041
5,1,1263,2.0,1260759151,0.887041
6,1,1287,2.0,1260759187,0.887041
7,1,1293,2.0,1260759148,0.887041
8,1,1339,3.5,1260759125,0.887041
9,1,1343,2.0,1260759131,0.887041


In [159]:
df_ratings.head(3)

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182


In [171]:
#df_std_rating

In [146]:
df_ratings[df_ratings.userId==27].std()

userId          0.000000
movieId       952.966259
rating          1.029217
timestamp    1797.440162
dtype: float64

In [135]:

df_ratings = get_data()
df_ratings_viewcount = get_benchmark_feature(df_ratings)
print (df_ratings_viewcount)
recommend_movie = recommend(df_ratings_viewcount)



        userId  movieId  rating   timestamp  total_view  avg_rating
0            1       31     2.5  1260759144          42    3.178571
1            7       31     3.0   851868750          42    3.178571
2           31       31     4.0  1273541953          42    3.178571
3           32       31     4.0   834828440          42    3.178571
4           36       31     3.0   847057202          42    3.178571
5           39       31     3.0   832525157          42    3.178571
6           73       31     3.5  1255591860          42    3.178571
7           88       31     3.0  1239755559          42    3.178571
8           96       31     2.5  1223256331          42    3.178571
9          110       31     4.0   840100695          42    3.178571
10         111       31     3.5  1097429230          42    3.178571
11         150       31     2.5  1130905954          42    3.178571
12         161       31     3.0   837629820          42    3.178571
13         165       31     3.5  1111981801     